# Imports

In [1]:
import os
import numpy as np
os.environ["SAGE_NUM_THREADS"] = '64' #set to number of cores

In [2]:
from conservation_laws.polynomial_conserved_functions import get_coef_equations

to create a generic polynomial with 𝐷 indeterminates with generic coefs

In [2]:
#run get_all_polynomial_conserved_functions.ipynb
from conservation_laws.polynomial_conserved_functions import *

to get all polynomial conservation laws for given vector fields

In [1]:
#run get_specific_vector_fields_and_known_conserved_functions.ipynb
from conservation_laws.known_conserved_functions import *
from conservation_laws.nn_vector_fields import *

to get 
- the vector fields asociated to linear and ReLU neural networs and 
- the number of independent conserved functions already known in these cases

In [5]:
#run computation_dimension_trace_Lie_algebra.ipynb # to get the dim of Lie(W)(\theta)
from conservation_laws.lie import *

In [6]:
sage.parallel.ncpus.ncpus()

64

In [7]:
# increase the maximum memory usage
# warning: this can consume a lot of RAM
maxima._eval_line(":lisp (ext:set-limit 'ext:heap-size 100000000000)",
wait_for_prompt=False)

# Test on specific vector fields $v_i$ (e.g. $v_i:= \nabla \phi_i$).
**In this notebook we test our results on different vector fields ($(v_i)_{i = 1, \cdots, d}$), from simple examples to more sophisticated ones. For ReLU and linear neural networks, the vector fields ($(v_i)_{i = 1, \cdots, d}$) associated to the model $\phi$ (then $v_i = \nabla \phi_i$) are build in the notebook run get_specific_vector_fields_and_known_conserved_functions.ipynb, as well as the number $N$ of independent conservation laws already known by the literature. We systemically compute $\mathrm{dim} \mathrm{Lie} W_\phi (\theta)$ (via the notebook computation_dimension_trace_Lie_algebra.ipynb) and we check if $D- \mathrm{dim} \mathrm{Lie} W_\phi (\theta) = N$: if so, we know thanks to Theorem 3.3 that there are no more conserved functions. Finally we check that all polynomial conservation laws we get (via the notebook get_all_polynomial_conserved_functions.ipynb) correspond exactly to the known conservation laws, for ReLU and linear cases.**

### Table of contents: 
- **Part A: Test when $(v_i)_i$ are random polynomial vector fields**  
- **Part B: Test on the simple case: $\phi(\theta) = \theta_1 \times \cdots \times \theta_D$**  
- **Part C: Test on the $q$-layer linear networks (see Section 4 of our paper)**  
- **Part D: Test on the $q$-layer ReLU networks (with or without bias)** 



## A. Test on $d$ random polynomial of degree $m$ vector fields.

In [8]:
def build_vector_fields(D, d, m):
    list_var = [var('x'+str(i+1)) for i in range(D)]
    R = PolynomialRing(ZZ, list_var) 
    return np.array([[R.random_element(degree = m) for _ in range(D)] for _ in range(d)]).T

In [9]:
D = 5
random_vector_fields = build_vector_fields(D, 1, 3)
get_conserved_polynomials(0, random_vector_fields, True) # the kernel dimension is 1 but the constant polynomial is not counted because
# its gradient is zero so is not a linearly independent vector

# the first argument of solve is the degree of conserved poynomial we are looking for

([1.00000000000000], 0)

In [10]:
get_conserved_polynomials(4, random_vector_fields, True)  # in general, there are no polynomial conserved functions for random polynomial vector fields

0

In [11]:
D - dim_trace_lie_algebra(random_vector_fields, fast=False)  # number of independent conserved laws in that case

4

## B. Test on $\phi(\theta) = \theta_1 \times \cdots \times \theta_D$ (*i.e.*  $d = 1$ and $v(\theta)= \nabla \phi (\theta)$).

In [12]:
def particular_vf(D):
    liste_var = [var('x'+str(i+1)) for i in range(D)]
    R = PolynomialRing(ZZ, liste_var) 
    list_ = [1]*D
    Q = R.monomial(*list_)
    return np.array(Q.gradient()).reshape(D, 1)

In [13]:
particular_vf(3)

array([[x2*x3],
       [x1*x3],
       [x1*x2]], dtype=object)

We obtain that there are $D-1$ independent homogeneous polynomial of degree 2 conserved functions (and there are no more conserved functions as $(D-1)+ 1 = D$).

In [14]:
V2 = particular_vf(3)
get_conserved_polynomials(2, V2, True) 

([-x1^2 + x3^2, -x1^2 + x2^2], 2)

Let's check that the dimension of $Lie(W_\phi)(\theta)$ is equal to 1 !

In [15]:
dim_trace_lie_algebra(V2, fast=False)

1

## C. Test on the case of $q$-layer linear neural networks $\phi(U_1, \cdots, U_q) = U_1 \times \cdots U_q$ ($v_i:= \nabla \phi_i(U_1, \cdots, U_q)$).

In [16]:
from tqdm import tqdm

In [17]:
np.random.seed(0)
for i in tqdm(range(50)):
    depth = np.random.randint(2, 5)
    list_dim = np.empty(depth + 1, dtype=int)
    for j in range(depth + 1):
        list_dim[j] = np.random.randint(2, 6)
    list_dim = list(list_dim)
    n_known_functions = known_conserved_functions_linear(list_dim)
    D, vector_fields = vector_fields_for_q_layer_LNN(list_dim)
    dim_found = D - dim_trace_lie_algebra(vector_fields, n_known_functions=n_known_functions, verbose=0)
    assert dim_found == n_known_functions, "dim_found = {}, n_known_functions = {}, for list_dim {}".format(dim_found, n_known_functions, list_dim)


  0%|          | 0/50 [00:00<?, ?it/s]

 12%|█▏        | 6/50 [00:41<05:02,  6.86s/it]


KeyboardInterrupt: 

The number of independent conserved functions that are already known matches with $D - \text{dim} Lie W_{\phi} (\theta)$: there is no other conservation law!

In [ ]:
np.random.seed(0)
for i in tqdm(range(10)):
    depth = np.random.randint(2, 4)
    list_dim = np.empty(depth + 1, dtype=int)
    for j in range(depth + 1):
        list_dim[j] = np.random.randint(2, 5)
    list_dim = list(list_dim)
    D, vector_fields = vector_fields_for_q_layer_LNN(list_dim)
    n_known_functions = known_conserved_functions_linear(list_dim)
    assert solve(2, vector_fields) == n_known_functions

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:31<00:00,  9.16s/it]


As expected, we can also find all known conserved functions by looking for polynomial conserved functions of degree 2, using our algorithm presented in Section 2.5 of our paper.

## D. Test on the case of $q$-layer ReLU neural networks.

In [18]:
np.random.seed(0)
for i in tqdm(range(50)):
    depth = np.random.randint(2, 5)
    list_dim = np.empty(depth + 1, dtype=int)
    for j in range(depth + 1):
        list_dim[j] = np.random.randint(2, 10)
    list_dim = list(list_dim)
    if np.random.random() < 0.5:
        bias = True
    else:
        bias = False
    n_known_functions = known_conserved_functions_ReLU(list_dim)
    D, vector_fields = vector_fields_ReLU_q_layers(list_dim, bias=bias)
    dim_found = D - dim_trace_lie_algebra(vector_fields, n_known_functions=n_known_functions, verbose=0)
    assert dim_found == n_known_functions, "dim_found = {}, n_known_functions = {}, for list_dim {}".format(dim_found, n_known_functions, list_dim)


  2%|▏         | 1/50 [00:01<01:01,  1.25s/it]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x10d3d72e0>>
Traceback (most recent call last):
  File "/Users/smarcotte/opt/anaconda3/envs/abide-the-flow/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
 10%|█         | 5/50 [03:50<30:19, 40.44s/it]  

The number of independent conserved functions that are already known matches with $D - \text{dim} Lie W_{\phi} (\theta)$: there is no other conservation law!

In [ ]:
np.random.seed(0)
for i in tqdm(range(10)):
    depth = np.random.randint(2, 4)
    list_dim = np.empty(depth + 1, dtype=int)
    for j in range(depth + 1):
        list_dim[j] = np.random.randint(2, 5)
    list_dim = list(list_dim)
    n_known_functions = known_conserved_functions_ReLU(list_dim)
    D, vector_fields = vector_fields_ReLU_q_layers(list_dim)
    assert solve(2, vector_fields) == n_known_functions


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:44<00:00,  4.49s/it]


As expected, we can also find all known conserved functions by looking for polynomial conserved functions of degree 2, using our algorithm presented in Section 2.5 of our paper.